# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data_df = pd.read_csv("../output_data/cities.csv")
data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Sayyan,15.1718,44.3244,62.10,70,75,0.67,YE,1627169499
1,Lakes Entrance,-37.8811,147.9810,51.19,54,67,16.08,AU,1627169504
2,Yellowknife,62.4560,-114.3525,58.32,44,75,1.99,CA,1627169414
3,Dikson,73.5069,80.5464,37.60,96,94,12.17,RU,1627169487
4,Rikitea,-23.1203,-134.9692,69.46,55,0,1.25,PF,1627169505
...,...,...,...,...,...,...,...,...,...
573,Saint-Tite,46.7334,-72.5658,75.51,73,91,4.83,CA,1627169717
574,La Ronge,55.1001,-105.2842,75.45,41,75,11.50,CA,1627169497
575,Begamganj,23.6000,78.3333,74.34,96,98,7.20,IN,1627169718
576,Zhigansk,66.7697,123.3711,63.12,59,26,17.27,RU,1627169718


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
locations = data_df[["Lat", "Lng"]]

humidity = data_df["Humidity"].astype(float)

In [20]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

heatmap = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = humidity.max(), point_radius = 2)

fig.add_layer(heatmap)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

* A max temperature lower than 80 degrees but higher than 70.
* Wind speed less than 10 mph.
* Zero cloudiness.
* Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

In [5]:
new_df = data_df.loc[(data_df["Max Temp"] <= 80) & (data_df["Max Temp"] >= 70) & 
                     (data_df["Wind Speed"] <=10) & (data_df["Cloudiness"] == 0)]

new_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,Natal,-5.7950,-35.2094,77.22,73,0,9.22,BR,1627169450
34,Kemin,42.7846,75.6892,71.96,27,0,5.93,KG,1627169515
121,Mackay,-21.1500,149.2000,73.04,42,0,1.14,AU,1627169482
130,Plátanos,37.6667,21.6167,77.68,62,0,3.91,GR,1627169549
181,Marsaxlokk,35.8419,14.5431,79.09,64,0,5.75,MT,1627169569
216,Şırnak,37.5228,42.4594,71.06,28,0,4.97,TR,1627169583
226,Ponta do Sol,32.6667,-17.1000,73.71,77,0,6.08,PT,1627169481
244,Mizdah,31.4451,12.9801,73.71,46,0,9.35,LY,1627169593
271,Oytal,42.9057,73.2697,75.47,27,0,3.87,KZ,1627169603
339,Ubari,26.5921,12.7805,79.75,22,0,7.09,LY,1627169628


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = new_df.loc[:, ('City', 'Country', 'Lat', 'Lng') ].reset_index(drop = True)

hotel_df["Hotel Name"] = ""

hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
0,Natal,BR,-5.7950,-35.2094,
1,Kemin,KG,42.7846,75.6892,
2,Mackay,AU,-21.1500,149.2000,
3,Plátanos,GR,37.6667,21.6167,
4,Marsaxlokk,MT,35.8419,14.5431,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "",
    "radius": 5000,
    "type": "hotel",
    "keyword": "hotel",
    "key": g_key,
}

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    name = row["City"]
    params["location"] = f"{lat},{lng}"

    response = requests.get(base_url, params=params).json()
    print(f"Searching for city number {index + 1}: {name}")
    
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        print(f"Closest hotel in {name} is: {response['results'][0]['name']}")
        print("------------------")

    except (KeyError, IndexError):
        print(f"Missing result for City: {name}.")
        print("------------------")
        

print("-----------------------------")
print("Data Retrieval Complete")
print("-----------------------------")    

Searching for city number 1: Natal
Closest hotel in Natal is: eSuites VILA DO MAR Natal
------------------
Searching for city number 2: Kemin
Closest hotel in Kemin is: Gostinitsa
------------------
Searching for city number 3: Mackay
Closest hotel in Mackay is: Ocean International Hotel
------------------
Searching for city number 4: Plátanos
Closest hotel in Plátanos is: Hotel Europa Olympia
------------------
Searching for city number 5: Marsaxlokk
Closest hotel in Marsaxlokk is: Cugó Gran Macina Grand Harbour
------------------
Searching for city number 6: Şırnak
Closest hotel in Şırnak is: Şehr-i Nuh Otel
------------------
Searching for city number 7: Ponta do Sol
Closest hotel in Ponta do Sol is: Palace of Zinos
------------------
Searching for city number 8: Mizdah
Closest hotel in Mizdah is: فندق مزدة الكبير
------------------
Searching for city number 9: Oytal
Closest hotel in Oytal is: Sanatorium Ak-Niet
------------------
Searching for city number 10: Ubari
Closest hotel in

In [8]:
hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != ""), :]
hotel_df
#hotel_df = hotel_df[["Hotel Name", "City", "Country", "Lat", "Lng"]]



,City,Country,Lat,Lng,Hotel Name
0,Natal,BR,-5.7950,-35.2094,eSuites VILA DO MAR Natal
1,Kemin,KG,42.7846,75.6892,Gostinitsa
2,Mackay,AU,-21.1500,149.2000,Ocean International Hotel
3,Plátanos,GR,37.6667,21.6167,Hotel Europa Olympia
4,Marsaxlokk,MT,35.8419,14.5431,Cugó Gran Macina Grand Harbour
5,Şırnak,TR,37.5228,42.4594,Şehr-i Nuh Otel
6,Ponta do Sol,PT,32.6667,-17.1000,Palace of Zinos
7,Mizdah,LY,31.4451,12.9801,فندق مزدة الكبير
8,Oytal,KZ,42.9057,73.2697,Sanatorium Ak-Niet
9,Ubari,LY,26.5921,12.7805,فندق أوباري


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

fig
# Display figure


Figure(layout=FigureLayout(height='420px'))